#### Текст задания.
Дан график занятости работника, состоящий из:

расписания рабочего дня (days)
и времени заявок (timeslot).
Нужно реализовать следующий функционал:

1. Найти все занятые промежутки для указанной даты.
2. Найти свободное время для заданной даты.
3. Вывести доступен ли заданный промежуток времени для заданной даты.
4. (*) Сделать функцию, которая найдет для указанной продолжительности заявки свободное время в графике.
Формат аргументов:

Дата - гггг-дд-мм
Время - ЧЧ:ММ
Данные по графику можно получить через эндпоинт: GET https://ofc-test-01.tspb.su/test-task/

Нужно реализовать решение задачи на Python 3.6+. Написать юнит-тесты для проверки функционала, а также небольшое описание по их запуску. Добавить обработку ошибок.

Приветствуется использование "чистого кода" и ООП.

Пример данных:

{
    "days": [
        {"id": 1, "date": "2024-10-10", "start": "09:00", "end": "18:00"},
        {"id": 2, "date": "2024-10-11", "start": "08:00", "end": "17:00"}
    ],
    "timeslots": [
        {"id": 1, "day_id": 1, "start": "11:00", "end": "12:00"},
        {"id": 3, "day_id": 2, "start": "09:30", "end": "16:00"}
    ]
}

#### Основной код для поставленых задач.

Функции <span style="color:#dcdcaa;">pretty_print</span> и <span style="color:#dcdcaa;">get_full_empls_data</span> были написаны для удобства вывода и получения данных с url, т.е. для личного пользования.


Остальные функции: <span style="color:#dcdcaa;">get_busy_time</span>, <span style="color:#dcdcaa;">get_free_slots</span>, <span style="color:#dcdcaa;">check_time</span> и <span style="color:#dcdcaa;">get_free_time</span> описывают поставленные задачи.

Так как в условиях было прописана рекомендация на использование ООП и "чистого кода", был создан основной класс <span style="color:#41c989;">Worktime</span> ( лучше названия не придумал:) ), а также использована библиотека <span style="color:#41c989;">typing</span>, которую в последнее время часто стараюсь использовать в разработке для формирования хорошей привычки.

In [1]:
import requests as r
import json
from typing import *
from datetime import datetime, timedelta

class Worktime:
    """ Класс функций для рабочего времени """
    def __init__(self, source_url):
        self.sourse_url: str = source_url
        self.empls_data: Dict = r.get(self.sourse_url).json()
        self.days: List = self.empls_data["days"]
        self.timeslots: List = self.empls_data["timeslots"]

    def pretty_print(self, value) -> Dict:
        """ Удобный вывод JSON """
        return json.dumps(value, indent=4)

    def get_full_empls_data(self) -> Dict:
        """ Получение расписания работчего дня """
        return self.empls_data

    def get_busy_time(self, date: str) -> List:
        """ Поиск занятых промежутков для указанной даты """

        busy_intervals = []
        current_date_info = [day for day in self.days if day["date"] == date][0]
        for slot in self.timeslots:
            if current_date_info["id"] == slot["day_id"]:
                busy_intervals.append({
                    "date": current_date_info["date"], 
                    "start": slot["start"],
                    "end": slot["end"],
                })

        # Сортировка слотов с начала дня
        busy_intervals = sorted(busy_intervals, key = lambda d: (datetime.strptime(d['date'], "%Y-%m-%d"), datetime.strptime(d['start'], "%H:%M")))
        return busy_intervals

    def get_free_slots(self, date: str) -> List:
        """ Поиск свободных промежутков для указанной даты """

        current_date_info = [day for day in self.days if day["date"] == date][0]
        busy_time = self.get_busy_time(date)

        start_time = current_date_info["start"]
        free_slots = []

        for busy_slot in busy_time:
            if start_time == busy_slot["start"]:
                start_time = busy_slot["end"]
            elif start_time < busy_slot["start"]:
                # Получение количества свободных часов в промежутке для функции get_free_time
                free_hours_cnt = datetime.strptime(busy_slot["start"], "%H:%M") - datetime.strptime(start_time, "%H:%M")
                hours, remainder = divmod(free_hours_cnt.seconds, 3600)
                minutes = remainder // 60

                free_slots.append({ "date": current_date_info["date"], "start": start_time, "end": busy_slot["start"], "free_hours": f"{hours:02d}:{minutes:02d}" })
                start_time = busy_slot["end"]
            
        return free_slots
    
    def check_time(self, date: str, start_time: str, end_time: str) -> bool:
        """ Проверка занятости интервала для указанной даты """

        free_time_slots = self.get_free_slots(date)

        start_time = datetime.strptime(start_time, "%H:%M")
        end_time = datetime.strptime(end_time, "%H:%M")

        for free_slot in free_time_slots:
            free_slot_start = datetime.strptime(free_slot["start"], "%H:%M")
            free_slot_end = datetime.strptime(free_slot["end"], "%H:%M")

            if start_time >= free_slot_start and end_time <= free_slot_end:
                return True
            else:
                pass

        return False

    def get_free_time(self, date: str, duration: str) -> Dict:
        """ Получение свободного слота по продолжительности заявки """

        duration = datetime.strptime(duration, "%H:%M")
        duration_td = timedelta(hours=duration.hour, minutes=duration.minute)
        free_slots = self.get_free_slots(date)

        for slot in free_slots:
            if duration <= datetime.strptime(slot["free_hours"], "%H:%M"):
                return {
                    "date": date,
                    "start": slot["start"],
                    "end": (datetime.strptime(slot["start"], "%H:%M") + duration_td).strftime("%H:%M"),
                    "description": "На сегодня есть свободный слот",
                    "result": True
                }
        
        return {
            "description": "На сегодня свободного слота нет",
            "result": False
        }


if __name__ == "__main__":
    worktime = Worktime("https://ofc-test-01.tspb.su/test-task/")

    # print(worktime.pretty_print(worktime.get_full_empls_data()))
    print("1 Задание. Вывод занятых промежутков.", end='\n')
    print(worktime.pretty_print(worktime.get_busy_time("2025-02-18")), end='\n\n\n')
    print("2 Задание. Вывод свободных промежутков.", end='\n')
    print(worktime.pretty_print(worktime.get_free_slots("2025-02-18")), end='\n\n\n')
    print("3 Задание. Проверка занятости промежутка.", end='\n')
    print(worktime.pretty_print(worktime.check_time("2025-02-18", "10:00", "11:30")))
    print(worktime.pretty_print(worktime.check_time("2025-02-18", "11:10", "11:30")), end='\n\n\n')
    print("4 Задание. Получение свободного слота.", end='\n')
    print(worktime.pretty_print(worktime.get_free_time("2025-02-18", "10:00")))
    print(worktime.pretty_print(worktime.get_free_time("2025-02-18", "01:00")), end='\n\n\n')
    

1 Задание. Вывод занятых промежутков.
[
    {
        "date": "2025-02-18",
        "start": "10:00",
        "end": "11:00"
    },
    {
        "date": "2025-02-18",
        "start": "11:30",
        "end": "14:00"
    },
    {
        "date": "2025-02-18",
        "start": "14:00",
        "end": "16:00"
    },
    {
        "date": "2025-02-18",
        "start": "17:00",
        "end": "18:00"
    }
]


2 Задание. Вывод свободных промежутков.
[
    {
        "date": "2025-02-18",
        "start": "11:00",
        "end": "11:30",
        "free_hours": "00:30"
    },
    {
        "date": "2025-02-18",
        "start": "16:00",
        "end": "17:00",
        "free_hours": "01:00"
    }
]


3 Задание. Проверка занятости промежутка.
false
true


4 Задание. Получение свободного слота.
{
    "description": "\u041d\u0430 \u0441\u0435\u0433\u043e\u0434\u043d\u044f \u0441\u0432\u043e\u0431\u043e\u0434\u043d\u043e\u0433\u043e \u0441\u043b\u043e\u0442\u0430 \u043d\u0435\u0442",
    "result":

#### Тесты

Ниже описывается запуск тестов.
Так как задания я выполнял в Jupiter Notebook, решил использовать <span style="color:#41c989;">ipytest</span> для запуска тестов прямо в .ipynb

In [2]:
import pytest
import ipytest

ipytest.autoconfig(raise_on_error=True)

Впрочем запуск тестов под Jupiter Notebook абсолютно прост - запуск кода который находится ниже.

То же самое можно сказать и про тестируемые функции, в название теста просто добавляется префикс <span style="color:#dcdcaa;">test_</span>.

Для вывода использовался флаг -vv, для более подробного вывода.

In [4]:
%%ipytest -vv

worktime = Worktime("https://ofc-test-01.tspb.su/test-task/")

def test_get_busy_time():
    """ Тестирование функции get_busy_time """

    assert worktime.get_busy_time("2025-02-18") == [
        {
            "date": "2025-02-18",
            "start": "10:00",
            "end": "11:00"
        },
        {
            "date": "2025-02-18",
            "start": "11:30",
            "end": "14:00"
        },
        {
            "date": "2025-02-18",
            "start": "14:00",
            "end": "16:00"
        },
        {
            "date": "2025-02-18",
            "start": "17:00",
            "end": "18:00"
        }
    ]

def test_get_free_slots():
    """ Тестирование функции get_free_slots """

    assert worktime.get_free_slots("2025-02-18") == [
        {
            "date": "2025-02-18",
            "start": "11:00",
            "end": "11:30",
            "free_hours": "00:30"
        },
        {
            "date": "2025-02-18",
            "start": "16:00",
            "end": "17:00",
            "free_hours": "01:00"
        }
    ]

def test_check_time():
    """ Тестирование функции check_time """

    assert worktime.check_time("2025-02-18", "10:00", "11:30") == False 

    # Провал теста
    #assert worktime.check_time("2025-02-18", "11:10", "11:30") == False  

def test_get_free_time():
    """ Тестирование функции get_free_time """

    '''
    assert worktime.get_free_time("2025-02-18", "10:00") == {
        "description": "\u041d\u0430 \u0441\u0435\u0433\u043e\u0434\u043d\u044f \u0441\u0432\u043e\u0431\u043e\u0434\u043d\u043e\u0433\u043e \u0441\u043b\u043e\u0442\u0430 \u043d\u0435\u0442",
        "result": false
    }    
    '''

    # Это тоже провальный тест
    assert worktime.get_free_time("2025-02-18", "10:00") == {
        "description": "\u041d\u0430 \u0441\u0435\u0433\u043e\u0434\u043d\u044f \u0441\u0432\u043e\u0431\u043e\u0434\u043d\u043e\u0433\u043e \u0441\u043b\u043e\u0442\u0430 \u043d\u0435\u0442",
        "result": true
    }


======================================= test session starts =======================================
platform win32 -- Python 3.10.9, pytest-8.4.1, pluggy-1.5.0 -- c:\Python\python.exe
cachedir: .pytest_cache
rootdir: d:\Programing\python\Новая папка
plugins: anyio-4.9.0, Faker-27.0.0, nbmake-1.5.5, time-machine-2.16.0
collecting ... collected 4 items

t_bb1a90e695b24c6093bb96249fe28d07.py::test_get_busy_time PASSED                             [ 25%]
t_bb1a90e695b24c6093bb96249fe28d07.py::test_get_free_slots PASSED                            [ 50%]
t_bb1a90e695b24c6093bb96249fe28d07.py::test_check_time PASSED                                [ 75%]
t_bb1a90e695b24c6093bb96249fe28d07.py::test_get_free_time FAILED                             [100%]

============================================ FAILURES =============================================
_______________________________________ test_get_free_time ________________________________________

    def test_get_free_time():
        """ Те

Error: ipytest failed with exit_code 1